In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-01-12T12:37:48.806610-05:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
import numpy as np
import pandas as pd
import gc

from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm


from time import time

In [4]:
%watermark --iversions

pandas: 1.3.5
numpy : 1.21.6



In [5]:
ID = 'id'
TARGET = 'target'

In [6]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [7]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [8]:
y_train = train[TARGET].ravel()

train.drop([ID, TARGET, 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_car_09_cat'], axis=1, inplace=True)
test.drop([ID, 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_car_09_cat'], axis=1, inplace=True)

print(f"{train.shape},{test.shape}")
      
ntrain = train.shape[0]

(595212, 51),(892816, 51)


In [11]:
%%time
train.to_csv('../input/x_train_drop_shap.csv.zip', index=False, compression='zip')
test.to_csv('../input/x_test_drop_shap.csv.zip', index=False, compression='zip')

CPU times: user 38.4 s, sys: 192 ms, total: 38.6 s
Wall time: 38.6 s


In [12]:
%%time
kf = KFold(5, shuffle=True, random_state=137)

for i, (train_index, test_index) in enumerate(kf.split(train,y_train)):

        print(i)
        xgtrain, xgval = train.loc[train_index], train.loc[test_index]
        y_tr, y_val = y_train[train_index], y_train[test_index]
        
        xgtrain['target'] = y_tr
        xgval['target'] = y_val
        
        xgtrain.to_csv(f'../input/xgtrain_fold_{i}_drop_shap.csv.gz', index=False, compression='gzip')
        xgval.to_csv(f'../input/xgval_fold_{i}_drop_shap.csv.gz', index=False, compression='gzip')

0
1
2
3
4
CPU times: user 4min 19s, sys: 356 ms, total: 4min 19s
Wall time: 4min 20s
